In [85]:
import numpy as np 
import pandas as pd
import portion as P 
import string
import itertools
import math
import logging
import re
from fractions import Fraction
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from functools import reduce


# Finding reductions for linear problems
Consider the following locally verifiable problem in (d, $`\delta`$)-biregular trees:
- $`\Sigma \subseteq [0,1]`$
- Task is to label the edges
- Sum of edge labels incident to
    - white nodes is $`\geq \alpha`$
    - black nodes is $`\leq \beta`$
- Additionally leaf nodes accept all possible neighborhoods.

This program tries to find a value $`\{a, b, ...\}`$ in each maximal separate continuous interval $`A, B, C, ... \subseteq \Sigma = [0, 1]`$, so that any valid labeling can be transformed to another valid labeling by **simultaneously** replacing all labels with the value corresponding to their interval. This reduces the set of labels down to $`\Sigma = \{a, b, ...\}`$.





The basic idea is to check every combination of these intervals and note which of these combinations could be neighborhoods of white and/or black nodes. Using these combinations we can form a system of linear inequalities that the simultanious reductions must satisfy. Lastly we use linear programming to search for a set of values.


## Example 1:

Locally verifiable problem $`\Pi`$ in (3,3)-biregular trees:
- $`\Sigma = [0, 1/3) \cup (2/3, 1]`$
- $`\alpha = 1`$
- $`\beta = 2`$

Clearly there are no 0-round solutions, as $`[1/3, 2/3] \cap \Sigma = \emptyset`$. Let $`A= [0, 1/3)`$ and $`B= (2/3, 1]`$. The possible combinations for a neighborhood are:

- A, A, A 
- A, A, B 
- A, B, B 
- B, B, B

We can also calculate the range of the sum of any given neighborhood:

- $`A, A, A = \left[0, 1 \right)`$
- $`A, A, B = \left(\frac{2}{3}, \frac{5}{3}\right)`$
- $`A, B, B = \left(\frac{4}{3}, \frac{7}{3}\right)`$
- $`B, B, B = \left(2, 3\right]`$

This table shows us, for example, that that the neighbourhood $`A, A, B`$ can be a neighbourhood of a black or a white node. This however doesn't yet mean that **any** three values from the corresponding ranges can surround both white and black nodes. It just means that if we were to collapse these ranges into single values, that collapsed neighbourhood would need to be suitable for both white and black nodes. Using this idea we can create a system of inequalities that any suitable reduction must satisfy.

```math
\begin{aligned}

a \in A \iff 0\leq a &< \frac{1}{3} \\
b \in B \iff \frac{2}{3} < b &\leq 1 \\
3a &\leq 2 \\
1 \leq 2a+b  &\leq 2 \\
1 \leq a+2b &\leq 2 \\
3b &\geq 1
\end{aligned}
```
Now any pair $`(a,b)`$ satisfying these inequalities will be a valid reducion. Using linear programming, we can find for example the solution $`(0.161667, 0.676667)`$. Now the problem just as easy as the LCL problem (in RE syntax):
```
A A A
A A B
A B B


A A B
A B B
B B B
```

Note that these reductions **cannot** be found separately: if we choose any value for $`a\in A`$, we can find values $`b_1, b_2, b_3 \in B`$ so that a white node has the neighbourhood $`a+a+b_1<1`$ or a black node has the neighbourhood $`a+b_2+b_3>2`$, which breaks our labeling. Same logic applies to fixing $`b`$ first.

In [86]:
# Define some variables

# Degrees of black and white nodes
d = 3
delta = 3

# The treshold for black and white sums
beta = Fraction(15, 10)
alpha = Fraction(10, 10)

# The set of possible labels
Sigma_string = "[0, 1/3) U (5/10, 4/5) U (4/5, 1]"

# Do you want to split Sigma into smaller parts? This can help in some situations.
do_split = False
split_count = 6

# Value used to handle strict inequalities in calculations. Can affect possible results. (We approximate a<b  <=> a<=b-epsilon <=> b>= a+epsilon.)
epsilon = 0.000001

# Create logger. Logging levels are: DEBUG, INFO, WARNING, ERROR, CRITICAL (not all are necessarily used) 
logger = logging.getLogger("log")
logger.setLevel(logging.DEBUG)
logger.propagate = False
formatter = logging.Formatter('%(levelname)s: %(message)s')

# Create console handler
ch = logging.StreamHandler()
ch.setFormatter(formatter)

logger.handlers = []
logger.addHandler(ch)


In [87]:
# Define parser for Sigma
def convert(s):
    try:
        return float(s)
    except ValueError:
        return (s)

params = {
    'disj': ' U '
}

interval_li = list(map(lambda x: P.from_string(x, conv=Fraction), Sigma_string.split(" U ")))
Sigma = reduce(lambda a, b: a | b, interval_li)        

logger.info(f"Sigma: {Sigma}")

INFO: Sigma: [Fraction(0, 1),Fraction(1, 3)) | (Fraction(1, 2),Fraction(4, 5)) | (Fraction(4, 5),Fraction(1, 1)]


In [88]:
def interval_list_splitter(interval_list, splits):
    # Split the interval list at (1/splits, 2/splits, ... , (splits-1)/splits)
    # This can help in some cases, especially when beta<alpha (anti-slack).
    # TODO: fix splitted interval endpoints, now both splits contain the endpoint. Can have a huge effect.
    # Now implemented as (] until "midpoint" and [) onwards. This is some handwavy variable, there may be some better way to determine these. 
    new_int_li = []
    
    midpoint = np.mean([alpha, beta])/np.mean([d, delta])

    for x in range(splits):

        left_in = x==0 or x/splits > midpoint
        
        right_in = x==splits-1 or (x+1)/splits <= midpoint

        if left_in and right_in:
            interval = P.closed(Fraction(x, splits), Fraction(x+1, splits)) & Sigma
        if left_in and not right_in:
            interval = P.closedopen(Fraction(x, splits), Fraction(x+1, splits)) & Sigma
        if not left_in and right_in:
            interval = P.openclosed(Fraction(x, splits), Fraction(x+1, splits)) & Sigma
        if not left_in and not right_in:
            interval = P.open(Fraction(x, splits), Fraction(x+1, splits)) & Sigma
        
        if not interval.empty: new_int_li.extend(interval)
    
    # Verify that splitting didn't create or miss any points:
    splitted_sigma = reduce(lambda a, b: a | b, new_int_li)
    if splitted_sigma != Sigma:
        logger.error("Interval splitter dropped/added some point(s)!")
        logger.error(f"Sigma:\n{Sigma}\nUnion of splitted intervals:\n{splitted_sigma}")

    logger.debug(f"Interval was split at every 1/{splits} fraction. The midpoint heuristic value was {midpoint}.")
    return new_int_li

In [89]:
def create_interval_df(interval_list):
    # Create the dataframe containing the list of intervals, their name (A, B, ...) and their possible future reduction. 

    interval_count = len(interval_list)
    
    intervals = pd.DataFrame({"interval": [x for x in interval_list]}, index=list(string.ascii_uppercase[0:interval_count]))
    intervals["reduction"] = None

    logger.info(f"Created following interval dataframe:\n{intervals}")
    return intervals, interval_count



In [90]:
def sum_intervals(combination, interval_list):
    # Basic_sum_intervals can only handle a list of atomic intervals, eg. not [[1,2], (3,4) U (4,5)].
    # This will translate a sum of more complicated intervals to atomic summations, while also considering the intervals that we are adding.

    # For example, [0, 1] U [5,6] + [0, 1] U [5,6] is not [0, 2] U [5, 7] U [10, 12], as they are the same original interval and thus must be discretized to the same value, so [5, 7] is not possible. 

    ints = []
    c = 0
    # Combination is some neighborhood eg. AAABBC, which is grouped to (A, (A,A,A)), (B, (B,B)), (C, (C))
    # Basically multiply the boundaries of all atomic intervals of a single interval by the number it's repeated in the neighborhood.
    # After that add that new multiplied interval to a list of different intervals, that are then combined and summed (cartesian product). 
    for i, g in itertools.groupby(combination):
        count = len("".join(g))
        intervals = [interval.apply(lambda x: (x.left, x.lower*count, x.upper*count, x.right)) for interval in interval_list[c]]
        c += count
        ints.append(intervals)

    return reduce(lambda a, b: a | b, map(lambda li: basic_sum_intervals(li), itertools.product(*ints)))

In [91]:
def basic_sum_intervals(interval_list):
    # Given a list of atomic intervals, determine the sum of those intervals, eg. the interval where the sum could end up to.
    
    min = 0
    max = 0
    min_in = True
    max_in = True
    for interval in interval_list:
        min_in = min_in and interval.left == P.CLOSED
        max_in = max_in and interval.right == P.CLOSED
        min += interval.lower
        max += interval.upper
    if min_in and max_in:
        return P.closed(min, max)
    if min_in and not max_in:
        return P.closedopen(min, max)
    if not min_in and max_in:
        return P.openclosed(min, max)
    return P.open(min, max)


In [92]:
def create_neighborhoods(intervals):
    # Create all of the possible neighborhoods that could exist, and check which of them could surround active or passive nodes (B/W).
    # If d =/= delta, it's not possible for a neighborhood to be suitable for both nodes.

    # First create all neighborhoods.
    if d==delta:
        combinations = list(itertools.combinations_with_replacement(intervals.index, d))
        neighborhoods = pd.DataFrame({"combination": combinations})
        neighborhoods["W"] = None 
        neighborhoods["B"] = None

    else:
        combinations = list(itertools.combinations_with_replacement(intervals.index, delta))
        combinations.extend(list(itertools.combinations_with_replacement(intervals.index, d)))
        neighborhoods = pd.DataFrame({"combination": combinations})
        neighborhoods["OK"] = None
        
    logger.debug(f"Created following neighborhood dataframe:\n{neighborhoods}")

    white_range = P.closedopen(alpha, P.inf)
    black_range = P.openclosed(-P.inf, beta)

    logger.debug(f"Black range: {black_range}")
    logger.debug(f"White range: {white_range}")
    logger.debug("\nRanges of neighbourhood sums")


    # Then calculate the intervals, that the sum of these neighborhoods could have. 
    # If they contain any values suitable for active/passive nodes, the neighbourhood is considered as suitable for that color.
    for index, row in neighborhoods.iterrows():
        interval = sum_intervals(row["combination"], list(map(lambda x: intervals.loc[x, "interval"], row["combination"])))
        
        c = " ".join(row["combination"])
        logger.debug(f'{c}: {interval}')

        # See if we have to check both white and black ranges or only one
        if d==delta:
            neighborhoods.at[index, "W"] = not (white_range & interval).empty
            neighborhoods.at[index, "B"] = not (black_range & interval).empty
            
        else: 
            if len(row["combination"]) == d:
                neighborhoods.at[index, "OK"] = not (black_range & interval).empty
                
            else: neighborhoods.at[index, "OK"] = not (white_range & interval).empty
            
    logger.info(f"Created following neighbourhoods:\n{neighborhoods}")
    return neighborhoods


In [93]:
def detect_unions(neighborhoods, intervals, interval_count):
    # Detect intervals in combinations-table that could be joined without breaking labelings.
    # This can happen in two cases: If two intervals have exactly the same rows, or rows of one interval is a strict subset of the other.
    # Using a single discretization value for them can enable discretization.
   
    # This is a brute force solution, any smarter ideas are welcome.
    
    # We can only look at neighborhoods with maximal amounts of either label of the pair, as labels are are absolutely ordered:
    #   Let A < B. Now A^nB^mXY is white --> A^(n-i)B^(m+i)XY is white for all 0<=i<=n. Same for black, except in the other direction.
    # Basically if the truth value for active-/passiveness of a neighborhood changes in middle of some series (AAA, AAB, ABB, BBB),
    # then the endpoints have different truth values. 
        
    pairs = [string.ascii_uppercase[i:i+2] for i in range(interval_count-1)]

    new_intervals = "A"

    for pair in pairs:

        low_worse = False
        high_worse = False
        problem_df = pd.DataFrame()
        
        others = [c for c in string.ascii_uppercase[0:interval_count] if c not in pair]

        for i in range(d, 0, -1):
            for end in itertools.combinations_with_replacement(others, d-i):

                low_index = tuple(sorted(tuple(itertools.repeat(pair[0], i)) + end))
                high_index = tuple(sorted(tuple(itertools.repeat(pair[1], i)) + end))
                
                low_row = neighborhoods.loc[neighborhoods["combination"] == low_index,  :]
                high_row = neighborhoods.loc[neighborhoods["combination"] == high_index, :]

                if d == delta:

                    if low_row["W"].iloc[0] != high_row["W"].iloc[0] and not low_worse:
                        low_worse = True
                        problem_df = problem_df.append(low_row, ignore_index=True)
                        problem_df = problem_df.append(high_row, ignore_index=True)

                    
                    if low_row["B"].iloc[0] != high_row["B"].iloc[0] and not high_worse:
                        high_worse = True
                        problem_df = problem_df.append(low_row, ignore_index=True)
                        problem_df = problem_df.append(high_row, ignore_index=True)
                    
                    if low_worse and high_worse:
                        logger.debug(f"{pair} is not interchangeable:\n{problem_df}")
                        break
                
                else:
                    if not low_row["OK"].iloc[0] and high_row["OK"].iloc[0] and not low_worse:
                        low_worse = True
                        problem_df.append(low_row, ignore_index=True)
                        problem_df.append(high_row, ignore_index=True)

                    
                    if low_row["OK"].iloc[0] and not high_row["OK"].iloc[0] and not high_worse:
                        high_worse = True
                        problem_df.append(low_row, ignore_index=True)
                        problem_df.append(high_row, ignore_index=True)
                    
                    if low_worse and high_worse:
                        logger.debug(f"{pair} is not interchangeable:\n{problem_df}")
                        # Breaks innermost loop (end)
                        break
            # Breaks outermost loop (i)
            if low_worse and high_worse: break 

        if d != delta:
            for i in range(delta, 0, -1):
                for end in itertools.combinations_with_replacement(others, delta-i):
                    low_index = tuple(sorted(tuple(itertools.repeat(pair[0], i)) + end))
                    high_index = tuple(sorted(tuple(itertools.repeat(pair[1], i)) + end))

                    low_row = neighborhoods.loc[neighborhoods["combination"] == low_index,  :]
                    high_row = neighborhoods.loc[neighborhoods["combination"] == high_index, :]
                    
                    if not low_row["OK"].iloc[0] and high_row["OK"].iloc[0] and not low_worse:
                        low_worse = True
                        problem_df = problem_df.append(low_row, ignore_index=True)
                        problem_df = problem_df.append(high_row, ignore_index=True)

                    
                    if low_row["OK"].iloc[0] and not high_row["OK"].iloc[0] and not high_worse:
                        high_worse = True
                        problem_df = problem_df.append(low_row, ignore_index=True)
                        problem_df = problem_df.append(high_row, ignore_index=True)
                    
                    if low_worse and high_worse:
                        logger.debug(f"{pair} is not interchangeable:\n{problem_df}")
                        break
        
        if low_worse and high_worse: new_intervals += " "
        new_intervals += pair[1]

    new_intervals = new_intervals.split(" ")

    # Are some intervals interchangeable (union needed)?
    return (len(new_intervals) == interval_count, new_intervals)

In [94]:
def join_intervals(intervals, union_list):
    # Create a new set of intervals after combining previous intervals, as directed by detect_unions.

    interval_list = []
    
    for chars in union_list:
        new_interval = intervals.at[chars[0], "interval"]
        for char in chars[1:]:
            new_interval = new_interval.union(intervals.at[char, "interval"])
        interval_list.append(new_interval)
    
    return interval_list

In [95]:
def print_retor(neighborhoods):
    # Translate the problem to round-eliminator formalism. 
    # Returned LCL is at most as hard as the given problem. 
    # If the problem is discretizable, the discretization gives a zero round mapping between these problems.

    if d == delta:
        white_retor = ""
        black_retor = ""
        for index, row in neighborhoods.iterrows():
            if row["W"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
            if row["B"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"

    else:
        white_retor = ""
        black_retor = ""
        for index, row in neighborhoods.iterrows():
            if len(row["combination"]) == d and row["OK"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"
            elif row["OK"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
    
    
    print("\nRound eliminator syntax:")
    print("\n"+black_retor)
    print("\n"+white_retor)

In [96]:
def find_reductions(neighborhoods, intervals, interval_count):
    # Find discrete values in given intervals that satisfy all constraints given by neighborhoods.
    # In practice the program creates a system of linear inequalities implied by interval definitions and neighborhoods in PULP
    # and gives that system to a solver, that then returns a solution, if one exists.


    model = LpProblem(name="Reductions", sense=LpMaximize)

    variables = dict(zip(list(string.ascii_lowercase[0:interval_count]), 
                    [LpVariable(name = symbol, lowBound=0, upBound=1) for symbol in list(string.ascii_lowercase[0:interval_count])]))
    
    

    # Add the constraints of the original sets
    for index, row in intervals.iterrows():
        variable = variables[index.lower()]
        interval = row["interval"]
        
        # Check for non-atomic intervals (caused by unions)
        if interval.atomic:
            if interval.left == P.CLOSED:
                model += (variable>=interval.lower, f"Init_{index}_lower")
            else:
                model += (variable>=interval.lower + epsilon, f"Init_{index}_lower")
                
            if interval.right == P.CLOSED:
                model += (variable<=interval.upper, f"Init_{index}_upper")
            else:
                model += (variable<=interval.upper - epsilon, f"Init_{index}_upper")
        

        # Non-atomic intervals need some tricks, as the value cannot satisfy all atomic interval conditions at the same time.
        # Tricks implemented as presented in https://download.aimms.com/aimms/download/manuals/AIMMS3OM_IntegerProgrammingTricks.pdf
        else: 
            atomics = len(interval)
            trick_var_count = math.ceil(math.log(atomics, 2))
            trick_vars = [LpVariable(name = f"Trick_var_{index}_{x}", lowBound=0, upBound=1, cat="Binary") for x in range(trick_var_count)]
            # Example: x in [0, 1] U [2, 3] U [4, 5] <=>
            # x >= 0 - 1000*(0 - trick_1) - 1000*(0 - trick_0)   &&   x <= 1 + 1000*(0 - trick_1) + 1000*(0 - trick_0)
            # x >= 2 - 1000*(0 - trick_1) - 1000*(0 - trick_0)   &&   x <= 3 + 1000*(0 - trick_1) + 1000*(1 - trick_0)
            # x >= 4 - 1000*(1 - trick_1) - 1000*(0 - trick_0)   &&   x <= 5 + 1000*(1 - trick_1) + 1000*(0 - trick_0)
            # 2 * trick_1 + trick_0 <= 2 


            # Atomic intervals are distinguished by binary representation of the interval number, eg. 1:st (0:th) atomic interval is chosen when all trick_vars are 0, etc.
            for c in range(atomics):
                tricksum = None
                c_binary = format(c, f'0{trick_var_count}b')[::-1]
                for i in range(trick_var_count):
                    tricksum += 1000*(int(c_binary[i])-trick_vars[i])

                if interval[c].left == P.CLOSED:
                    model += (variable>=interval[c].lower - tricksum, f"Init_{index}_lower_{c}")
                else:
                    model += (variable>=interval[c].lower + epsilon - tricksum, f"Init_{index}_lower_{c}")
                    
                if interval[c].right == P.CLOSED:
                    model += (variable<=interval[c].upper + tricksum, f"Init_{index}_upper_{c}")
                else:
                    model += (variable<=interval[c].upper - epsilon + tricksum, f"Init_{index}_upper_{c}")

    # Add the constraints imposed by summation of the original sets
    if d == delta:
        for index, row in neighborhoods.iterrows():
            if (row["W"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")
            if (row["B"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")

    else:  
        for index, row in neighborhoods.iterrows():
            if (len(row["combination"])==d) and row["OK"]:
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")
            elif (row["OK"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")


    logger.debug(f"\n{model}\n")

    # Search for solutions to the given system of inequalities
    if model.solve() == -1:
        print("No reductions found.")
    
    else:
        print("Reductions found: ")
        for var in model.variables():
            if "TRICK_VAR" not in var.name.upper() and "__DUMMY" not in var.name.upper():
                intervals.loc[var.name.upper(), "reduction"] = var.value()
            
            if "TRICK_VAR" in var.name.upper():
                logger.info(f"{var.name}: {var.value()}")
            
        
        print(intervals)
        print_retor(neighborhoods)
        


In [97]:
def run_reductor():
    # Main program running reductions. It first checks for 0-round solutions, then 

    # First check for 0-round solution
    easy_solution_interval = P.closed(Fraction(alpha, delta), Fraction(beta, d))
    easy_solutions = (easy_solution_interval & Sigma)
    if not easy_solutions.empty:
        print("0-round solution found.")
        print(f"Choose any single value from {P.to_string(easy_solutions, **params)}.")
    
    # Try to find some reductions
    else:
        print("No 0-round solutions found.")

        if do_split:
            interval_list = interval_list_splitter(interval_li, split_count)
        else:
            interval_list = interval_li
        intervals, interval_count = create_interval_df(interval_list) 
        
        neighborhoods = create_neighborhoods(intervals)
        ready, union_list = detect_unions(neighborhoods, intervals, interval_count)
        while not ready:
            logger.info(f"Joining intervals: {union_list}")
            interval_list = join_intervals(intervals, union_list)
            intervals, interval_count = create_interval_df(interval_list) 
            neighborhoods = create_neighborhoods(intervals)
            ready, union_list = detect_unions(neighborhoods, intervals, interval_count)
            


        find_reductions(neighborhoods, intervals, interval_count)
        

    


In [98]:
run_reductor()

INFO: Created following interval dataframe:
                          interval reduction
A  [Fraction(0, 1),Fraction(1, 3))      None
B  (Fraction(1, 2),Fraction(4, 5))      None
C  (Fraction(4, 5),Fraction(1, 1)]      None
DEBUG: Created following neighborhood dataframe:
  combination     W     B
0   (A, A, A)  None  None
1   (A, A, B)  None  None
2   (A, A, C)  None  None
3   (A, B, B)  None  None
4   (A, B, C)  None  None
5   (A, C, C)  None  None
6   (B, B, B)  None  None
7   (B, B, C)  None  None
8   (B, C, C)  None  None
9   (C, C, C)  None  None
DEBUG: Black range: (-inf,Fraction(3, 2)]
DEBUG: White range: [Fraction(1, 1),+inf)
DEBUG: 
Ranges of neighbourhood sums
DEBUG: A A A: [Fraction(0, 1),Fraction(1, 1))
DEBUG: A A B: (Fraction(1, 2),Fraction(22, 15))
DEBUG: A A C: (Fraction(4, 5),Fraction(5, 3))
DEBUG: A B B: (Fraction(1, 1),Fraction(29, 15))
DEBUG: A B C: (Fraction(13, 10),Fraction(32, 15))
DEBUG: A C C: (Fraction(8, 5),Fraction(7, 3))
DEBUG: B B B: (Fraction(3, 2),Fracti

No 0-round solutions found.


DEBUG: 
Reductions:
MAXIMIZE
None
SUBJECT TO
Init_A_lower: a >= 0

Init_A_upper: a <= 0.333332333333

Init_B_lower_0: - 1000 Trick_var_B_0 + b >= 0.500001

Init_B_upper_0: 1000 Trick_var_B_0 + b <= 0.799999

Init_B_lower_1: - 1000 Trick_var_B_0 + b >= -999.199999

Init_B_upper_1: 1000 Trick_var_B_0 + b <= 1001

Constraint_0_B: 3 a <= 1.5

Constraint_1_W: 2 a + b >= 1

Constraint_1_B: 2 a + b <= 1.5

Constraint_2_W: a + 2 b >= 1

Constraint_2_B: a + 2 b <= 1.5

Constraint_3_W: 3 b >= 1

VARIABLES
0 <= Trick_var_B_0 <= 1 Integer
a <= 1 Continuous
b <= 1 Continuous


INFO: Trick_var_B_0: 0.0


Reductions found: 
                                            interval reduction
A                    [Fraction(0, 1),Fraction(1, 3))  0.333332
B  (Fraction(1, 2),Fraction(4, 5)) | (Fraction(4,...  0.500001

Round eliminator syntax:

A A A
A A B
A B B


A A B
A B B
B B B

